# Teleoperation

In this example we'll control the Jetbot remotely with a gamepad controller connected to our web browser machine.

### Create gamepad controller

The first thing we want to do is create an instance of the ``Controller`` widget, which we'll use to drive our robot.
The ``Controller`` widget takes a ``index`` parameter, which specifies the number of the controller.  This is useful in case you
have multiple controllers attached, or some gamepads *appear* as multiple controllers.  To determine the index
of the controller you're using,

1. Visit [http://html5gamepad.com](http://html5gamepad.com).  
2. Press buttons on the gamepad you're using
3. Remember the ``index`` of the gamepad that is responding to the button presses

Next, we'll create and display our controller using that index.

In [1]:
import torch
print(torch.__version__)

1.6.0


In [3]:
import tensorflow
print(tensorflow.__version__)

2.4.0


In [4]:
import tensorflowgpu
print(tensorflow-gpu.__version__)

SyntaxError: invalid syntax (<ipython-input-4-7519ae193c7a>, line 1)

In [1]:
import ipywidgets.widgets as widgets
from jetcam.csi_camera import CSICamera
import cv2
from matplotlib import pyplot  as plt
#camera = CSICamera(width=224, height=224)

global data
data = []

global index
index = 112
controller = widgets.Controller(index=0)  # replace with index of your controller

display(controller)

Controller()

Even if the index is correct, you may see the text ``Connect gamepad and press any button``.  That's because the gamepad hasn't
registered with this notebook yet.  Press a button and you should see the gamepad widget appear above.

### Connect gamepad controller to robot motors

Now, even though we've connected our gamepad, we haven't yet attached the controls to our robot!  The first, and most simple control
we want to attach is the motor control.  We'll connect that to the left and right vertical axes using the ``dlink`` function.  The
``dlink`` function, unlike the ``link`` function, allows us to attach a transform between the ``source`` and ``target``.  Because
the controller axes are flipped from what we think is intuitive for the motor control, we'll use a small *lambda* function to
negate the value.

> WARNING: This next cell will move the robot if you touch the gamepad controller axes!

In [2]:
from jetracer.nvidia_racecar import NvidiaRacecar
import traitlets

car = NvidiaRacecar()

car.throttle_gain = 0.2

In [3]:
car.steering_offset=0.1
car.steering = 0

In [4]:
import traitlets
class someobj(traitlets.HasTraits):
    some = traitlets.Float()
    @traitlets.validate('some')
    def _x(self, proposal):
        global index
        if proposal['value'] == 1.0:
            camera = CSICamera(width=224, height=224)
            index+=1
            dir1 = '/media/jetson/data/lineImages/image' + str(index)+'.jpg' 
            row = []
            row.append(str(dir1))
            row.append(str(car.steering))
            row.append(str(car.throttle))
            image = camera.read()
            camera.cap.release()
            cv2.imwrite(dir1,image)
            global data
            data.append(row)
            print(row)
#             print(str(dir) +','+ str(car.steering)+','+ str(car.throttle))
class someobj2(traitlets.HasTraits):
    some = traitlets.Float()
    @traitlets.validate('some')
    def _x(self, proposal):
        print(proposal['value'] )
        global index
        if proposal['value'] == 1.0:
            car.throttle_gain = 0.2
        else: 
            car.throttle_gain = 0
        

In [6]:
left_link = traitlets.dlink((controller.axes[2], 'value'), (car, 'steering'), transform=lambda x: x)
right_link = traitlets.dlink((controller.axes[1], 'value'), (car, 'throttle'), transform=lambda x: -x)

someobj1 = someobj()
#someobj2 = someobj2()

camerabutton = traitlets.dlink((controller.buttons[7],'value'), (someobj1, 'some'))

#camerabutton = traitlets.dlink((controller.buttons[2],'value'), (car, 'throttle'), transform=lambda x: if x > 0: x = 0.2 else: x= 0)



In [7]:
import pandas as pd
labels = ['img','steering','throttle']
new_data = pd.DataFrame(columns=labels, data=data)
new_data.to_csv('/media/jetson/data/lineImages/data2.csv',index=False)
print(new_data)

                                            img              steering  \
0    /media/jetson/data/lineImages/image113.jpg                   0.0   
1    /media/jetson/data/lineImages/image114.jpg                   0.0   
2    /media/jetson/data/lineImages/image115.jpg  -0.19589830935001373   
3    /media/jetson/data/lineImages/image116.jpg  -0.18558305501937866   
4    /media/jetson/data/lineImages/image117.jpg   -0.4536271393299103   
5    /media/jetson/data/lineImages/image118.jpg                  -1.0   
6    /media/jetson/data/lineImages/image119.jpg   -0.6804406642913818   
7    /media/jetson/data/lineImages/image120.jpg                   0.0   
8    /media/jetson/data/lineImages/image121.jpg                   0.0   
9    /media/jetson/data/lineImages/image122.jpg    0.6185491681098938   
10   /media/jetson/data/lineImages/image123.jpg   -0.4536271393299103   
11   /media/jetson/data/lineImages/image124.jpg                   1.0   
12   /media/jetson/data/lineImages/image125.jpg    

Awesome! Our robot should now respond to our gamepad controller movements.  Now we want to view the live video feed from the camera!